In [54]:
%matplotlib inline
import SUGen as su
import numpy as np
from matplotlib import pyplot as plt

In [36]:
import PhysConst as pc
from scipy.io import savemat
from scipy.interpolate import interp1d

In [37]:
for i in range(3):
    print i

0
1
2


In [46]:
def sterileFraction(A):
    A = np.absolute(A)
    #print A
    #init_prod = np.asarray([[1.0/3,2.0/3,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0]])
    init_prod = np.asarray([[1.0/3,2.0/3,0,0],[1,0,0,0],[0,1,0,0],[0.5,0.5,0,0]])
    
    rowe = A[0,:]*A[0,:]
    #print rowe.shape
    rowmu = A[1,:]*A[1,:]
    rowtau = A[2,:]*A[2,:]
    rowSter = A[3,:]*A[3,:]
    
    mat2 = A*A
    termE = []
    termMu = []
    termTau = []
    termSter = []
    for i in range(4):
        termE.append(sum(mat2[i,:]*rowe))
        termMu.append(sum(mat2[i,:]*rowmu))
        termTau.append(sum(mat2[i,:]*rowtau))
        termSter.append(sum(mat2[i,:]*rowSter))
    
    
    #implementing this for testing purposes
    final_e = []
    final_mu = []
    final_tau = []
    final_ster = []
    #print np.asarray(termSter).shape
    #print np.asarray(termTau).shape
    for i in range(4):
        final_e.append(sum(init_prod[i,:]*termE))
        final_mu.append(sum(init_prod[i,:]*termMu))
        final_tau.append(sum(init_prod[i,:]*termTau))
        final_ster.append(sum(init_prod[i,:]*termSter))
        
    
    
    total = np.add(final_e, final_mu)
    total = np.add(total, final_tau)
    return final_e, final_mu, final_tau, final_ster

In [39]:
phyEl = pc.PhysicsConstants()
phyEl.numneu = 4
ndim = 3
count  = 0
U = su.SUGen(phyEl)


In [40]:
U.sample_params()
matrix = (U.matrix_gen())
# print matrix
# print matrix.conjugate()
# print matrix.conjugate().T
# print np.dot(matrix.conjugate().T,matrix)
a = sterileFraction(matrix)
print a
print b

[0.21776512868069253, 0.25630620649939001, 0.19849458977134382, 0.22740039813536692]
[0.20814183472507169, 0.32887842648162424, 0.14777353884679542, 0.23832598266420985]


In [41]:
def fractionFromUnitaryMatrix(A):
    init_prod = np.asarray([[1.0/3,2.0/3,0],[1,0,0],[0,1,0],[0,0,1]])
    
    rowe = A[0,:]*A[0,:]
    rowmu = A[1,:]*A[1,:]
    rowtau = A[2,:]*A[2,:]
    mat2 = A*A
    termE = []
    termMu = []
    termTau = []
    for i in range(3):
        termE.append(sum(mat2[i,:]*rowe))
        termMu.append(sum(mat2[i,:]*rowmu))
        termTau.append(sum(mat2[i,:]*rowtau))
    
    
    #implementing this for testing purposes
    final_e = []
    final_mu = []
    final_tau = []

    for i in range(4):
        final_e.append(sum(init_prod[i,:]*termE))
        final_mu.append(sum(init_prod[i,:]*termMu))
        final_tau.append(sum(init_prod[i,:]*termTau))
    
    
    total = np.add(final_e, final_mu)
    total = np.add(total, final_tau)
    return final_e, final_mu, final_tau, total
        
    

In [42]:
def processFile(filename):
    csv = np.genfromtxt(filename, delimiter=",")
    csv = csv[np.argsort(csv[:, 0])]
    #print csv.shape
    x = csv[:,0]
    index = []
    for i in range(len(x) - 1):
        if x[i] == x[i+1]:
            index.append(i)
    #index = index
    for i in reversed(index):
        csv = np.delete(csv,i,0)
    x = csv[:,0]
    y = csv[:,1]
    #print x.shape, y.shape
    f = interp1d(x,y)
    #f = interpld(x,y)
    return f

In [43]:
def dagger(U):
    return np.dot((U.conjugate().T),U)

In [44]:
def checkForCuts(mat,interpolators):
    m = mat.flatten()
    count = 0
    totalChi2 = 0
    chi2Limit = 10.42
    for i in m:
        try:
            temp = interpolators[count](i)
            if temp < -1:
                print temp, i, count
        except ValueError:
            return False
        count = count + 1
        totalChi2 = totalChi2 + temp
    
    if totalChi2 <= chi2Limit:
        print totalChi2
        return True
    else:
        return False
            
        
    

In [70]:
count = 0
frac_e = []
frac_mu = []
frac_tau = []
frac_ster = []
phyEl = pc.PhysicsConstants()
phyEl.numneu = 4
ndim = 3
count  = 0
U = su.SUGen(phyEl)
non_unitarity_frac = 0.9
interpolators = []
interpolators.append(processFile('ue1.csv'))
interpolators.append(processFile('ue2.csv'))
interpolators.append(processFile('ue3.csv'))
interpolators.append(processFile('umu1.csv'))
interpolators.append(processFile('umu2.csv'))
interpolators.append(processFile('umu3.csv'))
interpolators.append(processFile('utau1.csv'))
interpolators.append(processFile('utau2.csv'))
interpolators.append(processFile('utau3.csv'))
# fe2 = processFile('ue2.csv')
# fe3 = processFile('ue3.csv')
# fmu1 = processFile('umu1.csv')
# fmu2 = processFile('umu2.csv')
# fmu3 = processFile('umu3.csv')
# ftau1 = processFile('utau1.csv')
# ftau2 = processFile('utau2.csv')
# ftau3 = processFile('utau3.csv')
sterileFractionWithoutCuts = []
totalCount = 0
while True:
    totalCount = totalCount + 1
    U.sample_params()
    matrix = (U.matrix_gen())
    mat33 = matrix[0:3,0:3]
    #sterileFractionWithoutCuts.append(sterileFraction(matrix))
    I = np.identity(ndim)
    matDag = dagger(mat33)
    delta = np.subtract(matDag,I)
#     if count < 10:
#         print np.absolute(delta)
#         print mat33
#         count = count + 1
    if np.amax(np.absolute(delta)) <= non_unitarity_frac :
        flag = checkForCuts(np.absolute(mat33),interpolators)
        if flag == True:
            print "found one " + str(count)
            count = count + 1
            #efrac, mufrac, taufrac,total = fractionFromUnitaryMatrix(np.absolute(mat33))
            efrac, mufrac, taufrac,sterfrac = sterileFraction(np.absolute(matrix))
    #         frac_e.append(np.divide(efrac,total))
    #         frac_mu.append(np.divide(mufrac,total))
    #         frac_tau.append(np.divide(taufrac,total))
            frac_e.append(efrac)
            frac_mu.append(mufrac)
            frac_tau.append(taufrac)
            frac_ster.append(sterfrac)
        #print efrac, total
    
    if count > 1000:
        break

frac_e = np.asarray(frac_e)
frac_mu = np.asarray(frac_mu)
frac_tau = np.asarray(frac_tau)
#savemat('frac_90_ster_test.mat',mdict={'eFraction' : frac_e, 'muFrac':frac_mu, 'tauFrac':frac_tau, 'sterFrac' : frac_ster})
#savemat('frac_30_ster_test.mat',mdict={'sterFrac':frac_ster,'sterFracWithoutConstraint' : sterileFractionWithoutCuts})
    
print count, totalCount    

4.38620374704
found one 0
6.24434721368
found one 1
9.06713151156
found one 2
10.3823699748
found one 3
8.91815690822
found one 4
7.95201916815
found one 5
8.72409578464
found one 6
2.83460506073
found one 7
8.11962279267
found one 8
10.332073272
found one 9
8.17689806255
found one 10
9.68278867887
found one 11
8.64073053481
found one 12
9.12610713437
found one 13
7.21263235859
found one 14
7.18739989651
found one 15
9.87080067738
found one 16
7.71947375462
found one 17
6.41350929515
found one 18
4.91556156849
found one 19
5.97933513258
found one 20
6.91440863479
found one 21
8.55783011611
found one 22
4.40580655553
found one 23
9.82202575728
found one 24
9.50570080768
found one 25
6.51346152573
found one 26
7.52750733072
found one 27
9.49028127816
found one 28
8.02440055214
found one 29
9.3244518283
found one 30
9.84560550027
found one 31
3.93981024757
found one 32
8.17485818033
found one 33
8.44833952063
found one 34
9.91314632736
found one 35
10.4166656377
found one 36
9.25981792659

In [71]:
len(frac_mu)

1001

In [72]:
total = np.add(frac_e, frac_mu)
total = np.add(total,frac_tau)
total = np.add(total,frac_ster)

In [68]:
for i in total:
    print i,

[ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  1.  1.] [ 1.  1.  

In [60]:
prod120_ster = frac_ster[0]

In [73]:
np.max(np.asarray(frac_ster),axis=0)

array([ 0.30397076,  0.39164365,  0.44045473,  0.28936546])